<a href="https://colab.research.google.com/github/ApoorvaNagarajan/Image-Filter-Kernels/blob/master/assignment4/assignment4_iteration4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries and modules**

**Idea behind the network**

- Has three improvements over iteration3
- Improvements
  - Reduce number of parameters by moving the first max pooling payer one layer up
  - Adjust dropout rates
  - Adjust the learning rate

**Observations : Filled after running the network below. (Have put it in the top so that this is not missed out)**

- Parameters : 12,394
- Epochs : 20
- val_acc : 0.9940 ( at 8th epoch)
- Time taken for each epoch : 90us
- Did not see significant improvement in either parameters or accuracy


In [0]:
# Import all the required python libraries
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

from keras.datasets import mnist

### Load pre-shuffled MNIST data into train and test sets

In [0]:
# downloads the MNIST dataset and splits it into test and train.
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# X_train and X_test are reshaped from 3 dimensions to 4 dimensions
# 1st dim : num images
# 2nd and 3rd dim : Width and height of eaach of the images
# 4th dim : number of channels in each of the image. We set this to 1 as the
#           our dataset has only gray images
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)


# Normalizing the image
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [11]:
from keras.layers import Activation
model = Sequential()

 
model.add(Convolution2D(10, 3, 3, activation='relu', input_shape=(28,28,1))) # receptive field: 3x3, output dim: 26x26x10
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Convolution2D(16, 3, 3, activation='relu')) # receptive field: 5x5, output dim: 24x24x16
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(MaxPooling2D(pool_size=(2, 2))) # receptive field: 10x10, output dim: 12x12x16
model.add(Convolution2D(10, 1, 1, activation='relu'))# receptive field: 10x10, output dim: 12x12x10
model.add(BatchNormalization())

model.add(Convolution2D(16, 3, 3, activation='relu')) # receptive field: 12x12, output dim: 10x10x16
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Convolution2D(16, 3, 3, activation='relu')) # receptive field: 14x14, output dim: 8x8x16
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Convolution2D(10, 1, activation='relu')) # receptive field: 14x14, output dim: 8x8x10
model.add(BatchNormalization())
model.add(Convolution2D(10, 8)) # receptive field: 22x22, output dim: 1x1x10
model.add(Flatten())
model.add(Activation('softmax'))


model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (3, 3), activation="relu", input_shape=(28, 28, 1...)`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (1, 1), activation="relu")`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu")`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 26, 26, 10)        100       
_________________________________________________________________
batch_normalization_19 (Batc (None, 26, 26, 10)        40        
_________________________________________________________________
dropout_13 (Dropout)         (None, 26, 26, 10)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 24, 24, 16)        1456      
_________________________________________________________________
batch_normalization_20 (Batc (None, 24, 24, 16)        64        
_________________________________________________________________
dropout_14 (Dropout)         (None, 24, 24, 16)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 16)        0         
__________

In [0]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
def scheduler(epoch, lr):
  return round(0.005 * 1/(1 + 0.33 * epoch), 10)

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.005), metrics=['accuracy'])


In [27]:
model.fit(X_train, Y_train, batch_size=128, epochs=20, verbose=1, validation_data=(X_test, Y_test), callbacks=[LearningRateScheduler(scheduler, verbose=1)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20

Epoch 00001: LearningRateScheduler setting learning rate to 0.005.
60000/60000 [==============================] - 8s 141us/step - loss: 0.0288 - acc: 0.9905 - val_loss: 0.0285 - val_acc: 0.9911
Epoch 2/20

Epoch 00002: LearningRateScheduler setting learning rate to 0.0037593985.
60000/60000 [==============================] - 5s 91us/step - loss: 0.0237 - acc: 0.9921 - val_loss: 0.0367 - val_acc: 0.9874
Epoch 3/20

Epoch 00003: LearningRateScheduler setting learning rate to 0.0030120482.
60000/60000 [==============================] - 5s 90us/step - loss: 0.0203 - acc: 0.9933 - val_loss: 0.0231 - val_acc: 0.9924
Epoch 4/20

Epoch 00004: LearningRateScheduler setting learning rate to 0.0025125628.
60000/60000 [==============================] - 5s 90us/step - loss: 0.0200 - acc: 0.9931 - val_loss: 0.0219 - val_acc: 0.9928
Epoch 5/20

Epoch 00005: LearningRateScheduler setting learning rate to 0.0021551724.
60000/60000 [=========

In [28]:
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

[0.019796344216139187, 0.9935]
